In [1]:
import netCDF4 as nc
import scipy.interpolate as spi
import scipy.sparse as sp
import numpy as np
from salishsea_tools import nc_tools

In [10]:
def use_weights(weights,ops):
    """Given a weights and operation file, return a NEMO shaped array with wind data interpolated.
    
    :arg weights: Path to weights file to be used interpolation.
    :type weights: str
    
    :arg ops: Path to operational file to be used in interpolation.
    :type ops: str
    
    :returns: NEMO-sized Numpy array containing interpolations of wind data"""
    # Weights
    #weightsfile = '/home/mdunphy/MEOPAR/NEMO-forcing/grid/weights-gem2.5-ops.nc'
    with nc.Dataset(weights) as f:
        s1 = f.variables['src01'][:]-1  # minus one for fortran-to-python indexing
        s2 = f.variables['src02'][:]-1
        s3 = f.variables['src03'][:]-1
        s4 = f.variables['src04'][:]-1
        w1 = f.variables['wgt01'][:]
        w2 = f.variables['wgt02'][:]
        w3 = f.variables['wgt03'][:]
        w4 = f.variables['wgt04'][:]

    # Operational data
    #opsfile='/results/forcing/atmospheric/GEM2.5/operational/ops_y2017m04d29.nc'
    with nc.Dataset(ops) as f:
        odata = f.variables['tair'][0,...]   # Load a 2D field


    NO = odata.size   # number of operational grid points
    NN = s1.size      # number of NEMO grid points

    # Build matrix
    n = np.array([x for x in range(0,NN)])
    M1 = sp.csr_matrix((w1.flatten(), (n, s1.flatten())), (NN,NO))
    M2 = sp.csr_matrix((w2.flatten(), (n, s2.flatten())), (NN,NO))
    M3 = sp.csr_matrix((w3.flatten(), (n, s3.flatten())), (NN,NO))
    M4 = sp.csr_matrix((w4.flatten(), (n, s4.flatten())), (NN,NO))
    M = M1+M2+M3+M4

    # Interpolate by matrix multiply - quite fast
    ndata = M*odata.flatten()

    # Reshape to NEMO shaped array
    ndata=ndata.reshape(s1.shape)
    
    return ndata

In [11]:
opsfile='/results/forcing/atmospheric/GEM2.5/operational/ops_y2017m04d29.nc'
weightsfile ='/home/vdo/MEOPAR/NEMO-forcing/grid/weights-gem2.5-ops.nc'
ans = use_weights(weightsfile,opsfile)
print(ans)
ans.shape

[[ 285.18725039  285.21280022  285.26716202 ...,  275.34619208
   275.44375196  275.5399466 ]
 [ 285.18840503  285.19137598  285.24226713 ...,  275.66929409
   275.73676056  275.84787996]
 [ 285.19230722  285.17914618  285.21921623 ...,  275.96556107
   276.00473081  276.13578149]
 ..., 
 [ 278.04653682  277.66699859  277.30327173 ...,  273.95910365
   274.00692939  273.80333675]
 [ 277.9115867   277.60606786  277.34447418 ...,  273.69122196  273.6991445
   273.58461212]
 [ 278.04473504  277.78919863  277.54421604 ...,  273.458561    273.42629574
   273.3687499 ]]


(898, 398)

In [29]:
import netCDF4 as nc
import scipy.interpolate as spi
import scipy.sparse as sp
import numpy as np
from salishsea_tools import nc_tools

def build_matrix(weights,ops,variable):
    """Given a weights and operation file and a variable of the ops file, return matrix built with inputs    
    
    :arg weights: Path to weights file to be used interpolation.
    :type weights: str
    
    :arg ops: Path to operational file to be used in interpolation.
    :type ops: str
    
    :arg variable: Specified variable in ops file.
    :type variable: str
    
    :returns: SciPy Compressed Sparse Row matrix
    :type :py:class 'scipy.sparse.csr_matrix' """
    # Weights
    with nc.Dataset(weights) as f:
        s1 = f.variables['src01'][:]-1  # minus one for fortran-to-python indexing
        s2 = f.variables['src02'][:]-1
        s3 = f.variables['src03'][:]-1
        s4 = f.variables['src04'][:]-1
        w1 = f.variables['wgt01'][:]
        w2 = f.variables['wgt02'][:]
        w3 = f.variables['wgt03'][:]
        w4 = f.variables['wgt04'][:]

    # Operational data
    with nc.Dataset(ops) as f:
        odata = f.variables[variable][0,...]   # Load a 2D field


    NO = odata.size   # number of operational grid points
    NN = s1.size      # number of NEMO grid points

    # Build matrix
    n = np.array([x for x in range(0,NN)])
    M1 = sp.csr_matrix((w1.flatten(), (n, s1.flatten())), (NN,NO))
    M2 = sp.csr_matrix((w2.flatten(), (n, s2.flatten())), (NN,NO))
    M3 = sp.csr_matrix((w3.flatten(), (n, s3.flatten())), (NN,NO))
    M4 = sp.csr_matrix((w4.flatten(), (n, s4.flatten())), (NN,NO))
    M = M1+M2+M3+M4
    return M
    
    #######################################
    #new function begins here ish

def use_matrix(weights,ops,matrix,variable):
    """Given a weights file, opsfile, and a matrix, returns NEMO-shaped array interpolated with data 
    :arg weights: Path to weights file to be used interpolation.
    :type weights: str
    
    :arg ops: Path to operational file to be used in interpolation.
    :type ops: str
    
    :arg matrix: SciPy Compressed Sparse Row matrix
    :type :py:class 'scipy.sparse.csr_matrix'
    
    :arg variable: Specified variable in ops file.
    :type variable: str
    
    :returns: NEMO-sized Numpy array containing interpolations"""
    
    with nc.Dataset(weights) as f:
        s1 = f.variables['src01'][:]-1  # minus one for fortran-to-python indexing
    
    with nc.Dataset(ops) as f:
        odata = f.variables[variable][0,...]   # Load a 2D field
        
    # Interpolate by matrix multiply - quite fast
    ndata = matrix*odata.flatten()

    # Reshape to NEMO shaped array
    ndata=ndata.reshape(s1.shape)
    
    return ndata

In [30]:
ans2 = build_matrix(weightsfile,opsfile,'tair')
type(ans2)

scipy.sparse.csr.csr_matrix

In [31]:
ans3 = use_matrix(weightsfile,opsfile,ans2,'tair')
type(ans3)

numpy.ndarray